In [1]:
import numpy as np
import pandas as pd
import os
from random import shuffle
import cv2
import time
import seaborn as sns
import matplotlib.pyplot as plt
import glob as gb
%matplotlib inline


In [14]:
import keras
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,GlobalMaxPooling2D,AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.regularizers import l2
from keras.losses import categorical_crossentropy
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 

In [3]:
WIDTH=224
HEIGHT=224

In [4]:
train_image = "dataset/train"
test_image = "dataset/test"


print(train_image)
print(test_image)

dataset/train
dataset/test


In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   rotation_range=15,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [16]:
resnet50 = ResNet50(input_shape=(WIDTH,HEIGHT,3), weights='imagenet', include_top=False)

for layer in resnet50.layers:
  layer.trainable = False

x = resnet50.output
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(256, activation="relu")(x)

x = Dense(3, activation='softmax')(x)

model = Model(inputs=resnet50.input, outputs=x)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [17]:
from keras import backend as K
def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision
def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall
def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [18]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cvscores = []
for train, test in kfold.split(train_image):
    opt=Adam(learning_rate=0.001)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=[precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5),
                      'accuracy','AUC',])
    
    train_generator = train_datagen.flow_from_directory(train_image,
                                                 target_size = (WIDTH,HEIGHT),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

    test_generator = test_datagen.flow_from_directory(test_image,
                                            target_size = (WIDTH,HEIGHT),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            shuffle = False)
    
    start=time.time()
    history = model.fit_generator(train_generator,
                              validation_data=test_generator,
                              epochs=25) 
    print("training time: %.2f minutes"%((time.time()-start)/60))
    
    scores = model.evaluate(test_generator)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.
Epoch 1/25
322/322 [==============================] - 414s 1s/step - loss: 0.8797 - precision: 0.6564 - recall: 0.6167 - fbeta_score: 0.6467 - accuracy: 0.6476 - auc: 0.7969 - val_loss: 0.6921 - val_precision: 0.7718 - val_recall: 0.7153 - val_fbeta_score: 0.7588 - val_accuracy: 0.7570 - val_auc: 0.8816
Epoch 2/25
322/322 [==============================] - 414s 1s/step - loss: 0.7936 - precision: 0.6966 - recall: 0.5887 - fbeta_score: 0.6641 - accuracy: 0.6685 - auc: 0.8257 - val_loss: 0.6907 - val_precision: 0.6659 - val_recall: 0.6659 - val_fbeta_score: 0.6659 - val_accuracy: 0.6638 - val_auc: 0.8798
Epoch 3/25
322/322 [==============================] - 415s 1s/step - loss: 0.7336 - precision: 0.6861 - recall: 0.6254 - fbeta_score: 0.6636 - accuracy: 0.6645 - auc: 0.8452 - val_loss: 0.6519 - val_precision: 0.6667 - val_recall: 0.5586 - val_fbeta_score: 0.6403 - val_accuracy: 0.6638 - val_auc: 0.8742
E